# Práctica: Análisis comparativo de tecnologías usando RAG

Objetivo de la práctica

Cargar varios archivos de documentos de diferentes fuentes (por ejemplo, artículos científicos, noticias, informes técnicos) y generar embeddings para cada uno.

Implementar un retriever que busque documentos relevantes según consultas comparativas o de análisis.

Implementar un generator que combine los documentos recuperados y genere un resumen que permita comparar o evaluar la información.

Probar el sistema con preguntas de análisis crítico y comparativo.

1. Preparación de datos

- La carpeta Data contiene archivos .csv o .txt de diferentes fuentes sobre energía renovable (solar, eólica, almacenamiento).

- Generar embeddings para cada documento usando un modelo de embeddings (por ejemplo, text-embedding-3-small).

2. Implementación del Retriever

El retriever debe permitir:

- Buscar documentos relevantes por tema.

- Buscar documentos relevantes por comparación, es decir, identificar información sobre dos tecnologías o conceptos diferentes.

- Utilizar búsqueda semántica con embeddings para medir similitud.

3. Implementación del Generator

El generator debe:

   - Tomar los documentos recuperados y resumir los puntos clave.

   - Comparar tecnologías, métodos o resultados si la consulta lo solicita.

   - Generar un texto coherente y estructurado, por ejemplo:

       Introducción breve.

       Comparación de características, ventajas y desventajas.

       Conclusión resumida.

4. Consultas de prueba

- “Comparar eficiencia entre células solares de perovskita y de silicio.”

- “Ventajas y desventajas del almacenamiento en baterías vs. hidrógeno.”

- “Últimas innovaciones en energía eólica frente a energía solar.”